# Import Libraries

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes
# import folium

print("Libraries imported!")

Libraries imported!


# Download and Explore Data Set

In [3]:
!pip install beautifulsoup4
!pip install lxml
!pip install et_xmlfile
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

soup = BeautifulSoup(source, 'xml')

     |████████████████████████████████| 122kB 18.1MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 5.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built et-xmlfile


# Transform Data into a Pandas Dataframe

In [4]:
# Create pandas dataframe
table = soup.find("table")
table_rows = table.find_all("tr")

l = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighborhood"])
df

# Replace new lines, tabs, and carriage returns in fields
df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
df

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"


# Delete Rows Where Borough Is Not Assigned

In [34]:
df.drop(df[df["Borough"] == "Not assigned"].index, inplace=True)
df

df.drop(df.index[0], inplace=True)

df2 = df.reset_index(drop=True)
df2.head()

,PostalCode,Borough,Neighborhood
0,M5A,Downtown Toronto,"Regent Park, Harbourfront"
1,M6A,North York,"Lawrence Manor, Lawrence Heights"
2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
3,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
4,M1B,Scarborough,"Malvern, Rouge"


In [35]:
df2.shape

(101, 3)

# Add Latitude and Longitude to Dataframe

In [23]:
lat_long_df = pd.read_csv("Geospatial_Coordinates.csv")

full_df = pd.merge(df2, lat_long_df, on="PostalCode", how="inner")
full_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4A,North York,Victoria Village,43.725882,-79.315572
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
4,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242


# View Only Toronto Data

In [29]:
toronto_data = full_df[full_df["Borough"] == "Downtown Toronto"].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


# Create Map

In [33]:
import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

# Get geographical coordinates

address = "Toronto, ON, Canada"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# Add markers to map
for lat, lng, label in zip(toronto_data["Latitude"], toronto_data["Longitude"], toronto_data["Neighborhood"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         393 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5